In [1]:
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth
from pprint import pprint
import os

In [11]:
url = 'https://marsapi.ams.usda.gov/services/v1.2/reports'
username = os.getenv("USDA_MARS_API_KEY")


In [12]:
headers={'username':username}
resp = requests.get(url, auth = (username, ""))
raw_data = pd.DataFrame.from_dict(json.loads(resp.text))
raw_data['markets'] = raw_data['markets'].str[0]
raw_data['market_types'] = raw_data['market_types'].str[0]
filtered_df = raw_data[raw_data['market_types'] == "Terminal"]
filtered_df.to_csv('cache/terminal_markets.csv')

In [13]:
markets_df = filtered_df[['slug_id', 'report_title', 'markets']].rename(columns={'report_title': 'market_name'})
print(markets_df.head())


    slug_id                                        market_name  \
558    2277    Atlanta Terminal Market Fruit Prices (AJ_FV010)   
559    2278  Atlanta Terminal Market Vegetables Prices (AJ_...   
560    2279  Atlanta Terminal Market Onions and Potatoes Pr...   
561    2280     Atlanta Terminal Market Nuts Prices (AJ_FV040)   
562    2281  Baltimore Terminal Market Fruit Prices (BP_FV010)   

                       markets  
558    Atlanta Terminal Market  
559    Atlanta Terminal Market  
560    Atlanta Terminal Market  
561    Atlanta Terminal Market  
562  Baltimore Terminal Market  


In [15]:
markets_df['markets'] = markets_df['markets'].str.replace("Terminal Market", "")
for index, row in markets_df.iterrows():
    if row['slug_id'] == "3704":
        print("Dummy")
    if "Fruit" in row['market_name']:
        markets_df.at[index,'market_name'] = row['markets'] + "fruits"
    elif "Nuts" in row['market_name']:
        markets_df.at[index,'market_name'] = row['markets'] + "nuts"
    elif "Vegetables" in row['market_name'] or "Vegetable" in row['market_name']:
        markets_df.at[index,'market_name'] = row['markets'] + "vegetables"
    elif "Onions and Potatoes" in row['market_name']:
        markets_df.at[index, 'market_name'] = row['markets'] + "onions and potatoes"
    elif "Asian Vegetable" in row['market_name']:
        markets_df.at[index, 'market_name'] = row['markets'] + "asian vegetables"
    elif "Herbs" in row['market_name']:
        markets_df.at[index, 'market_name'] = row['markets'] + "herbs"
    elif "Tropical F&V" in row['market_name']:
        markets_df.at[index, 'market_name'] = row['markets'] + "tropical f&v"
    elif "Asian Vegetable" in row['market_name']:
        markets_df.at[index, 'market_name'] = row['markets'] + "asian vegetables"

markets_df.drop(columns=['markets'],inplace=True)
print(markets_df.head())

Dummy
    slug_id                  market_name
558    2277               Atlanta fruits
559    2278           Atlanta vegetables
560    2279  Atlanta onions and potatoes
561    2280                 Atlanta nuts
562    2281             Baltimore fruits


In [17]:
print(type(markets_df.iloc[0]['slug_id']))

<class 'str'>


In [34]:
markets_df.to_csv("markets_db.csv", index=False)

In [1]:
from usda_mmn_utils import *
prices_df = get_prices(commodity="Tomatoes", slug_id = "2282", start_year=2023)
print(prices_df.head())


  report_date  slug_id        variety                   package    item_size  \
0  12/29/2023     2282     VINE RIPES               15 lb flats          28s   
1  12/29/2023     2282     VINE RIPES  25 lb cartons place pack  extra large   
2  12/29/2023     2282     VINE RIPES  25 lb cartons place pack        jumbo   
3  12/29/2023     2282  MATURE GREENS       25 lb cartons loose     6x7 size   
4  12/29/2023     2282  MATURE GREENS       25 lb cartons loose     6x6 size   

       properties                   grade organic   origin low_price  \
0  GREEN-BREAKERS                  CD One       N   Canada     22.00   
1       LIGHT RED                     N/A       N  Florida     36.00   
2       LIGHT RED                     N/A       N  Florida     36.00   
3   LIGHT RED-RED  85% U.S. One or Better       N  Florida     28.00   
4   LIGHT RED-RED  85% U.S. One or Better       N  Florida     38.00   

  high_price unit_sales  
0      24.00        N/A  
1      38.00        N/A  
2      3

In [20]:
start_year = 2023
end_year=2023
date_str = "01/01/"+str(start_year)+":12/31/"+str(end_year)
filter_dict = {'commodity':"Tomatoes", 'report_begin_date':date_str}
filter_str = ""
for key in filter_dict.keys():
    filter_str = filter_str+key+"="
    filter_str = filter_str+filter_dict[key]+";"
print(filter_str)


commodity=Tomatoes;report_begin_date=01/01/2023:12/31/2023;


In [21]:

slug_id = 2282
url = "https://marsapi.ams.usda.gov/services/v1.2/reports"+"/"+str(slug_id)
username = os.getenv("USDA_MARS_API_KEY")   # You need to store your API key as a User Variable in Windows or store it in .bashrc 
url = url + "?q="+ filter_str + "&allSections=true"
response = requests.get(url, auth = (username, ""))
raw_data = json.loads(response.text)
raw_df = pd.DataFrame(raw_data[1]['results'])
print("Dummy")


https://marsapi.ams.usda.gov/services/v1.2/reports/2282?q=commodity=Tomatoes;report_begin_date=01/01/2023:12/31/2023;&allSections=true
Dummy


In [13]:
raw_df['origin'].unique()

array(['Florida', 'Canada', 'Mexico', 'Georgia', 'Maryland', 'Alabama',
       'Virginia', 'Tennessee', 'North Carolina', 'California',
       'Michigan', 'New York', 'Pennsylvania', 'New Jersey',
       'South Carolina', 'West Virginia', 'Guatemala'], dtype=object)

In [13]:
mystr = "This string is weird"
newstr = "_".join(mystr.split())
print(newstr)

This_string_is_weird
